In [104]:
import yfinance as yf
import pandas as pd
import time
from datetime import datetime

stocks = ['AMD', 'NVDA', 'META', 'TSLA']
start = datetime(2019, 2, 1)
end = datetime(2020, 4, 30)

stock_prices         = yf.download(stocks, start , end, auto_adjust = False)
stock_prices = stock_prices.resample('ME').last()
stock_prices.index   = stock_prices.index.tz_localize(None)      # change yf date format to match pdr
stock_prices         = stock_prices.filter(like='Adj Close')     # reduce to just columns with this in the name
stock_prices.columns = stock_prices.columns.get_level_values(1)  # tickers as col names, works no matter order of tics

stock_prices = stock_prices.unstack(level=0).reset_index()
stock_prices.columns = ['Ticker', 'Date', 'Adj Close']
stock_prices['Return'] = stock_prices.groupby('Ticker')['Adj Close'].pct_change()
stock_prices.set_index('Date', inplace=True) 
print(stock_prices.head(10))
stock_prices.to_csv("outputs/returns.csv", index=True)

[*********************100%***********************]  4 of 4 completed

           Ticker  Adj Close    Return
Date                                  
2019-02-28    AMD  23.530001       NaN
2019-03-31    AMD  25.520000  0.084573
2019-04-30    AMD  27.629999  0.082680
2019-05-31    AMD  27.410000 -0.007962
2019-06-30    AMD  30.370001  0.107990
2019-07-31    AMD  30.450001  0.002634
2019-08-31    AMD  31.450001  0.032841
2019-09-30    AMD  28.990000 -0.078219
2019-10-31    AMD  33.930000  0.170404
2019-11-30    AMD  39.150002  0.153846


In [105]:
pivot_data = stock_prices.pivot(columns='Ticker', values='Adj Close')
print(pivot_data)

Ticker            AMD        META      NVDA       TSLA
Date                                                  
2019-02-28  23.530001  160.692657  3.829050  21.325333
2019-03-31  25.520000  165.908066  4.457049  18.657333
2019-04-30  27.629999  192.492783  4.492793  15.912667
2019-05-31  27.410000  176.637527  3.366233  12.344000
2019-06-30  30.370001  192.094666  4.081179  14.897333
2019-07-31  30.450001  193.318878  4.192758  16.107332
2019-08-31  31.450001  184.799042  4.166809  15.040667
2019-09-30  28.990000  177.244644  4.329988  16.058001
2019-10-31  33.930000  190.750977  5.000369  20.994667
2019-11-30  39.150002  200.694138  5.395382  21.996000
2019-12-31  45.860001  204.287186  5.857403  27.888666
2020-01-31  47.000000  200.962875  5.885532  43.371334
2020-02-29  45.480000  191.567154  6.726964  44.532665
2020-03-31  45.480000  166.017548  6.565810  34.933334
2020-04-30  53.660000  193.279083  7.434112  53.367332


In [106]:
pairs = [('AMD', 'NVDA'), ('AMD', 'META')]

spread_list = []

for pair in pairs:
    ticker_1, ticker_2 = pair
    spread = pivot_data[ticker_1] - pivot_data[ticker_2]

    pair_df = spread.reset_index()
    pair_df['Date'] = pd.to_datetime(pair_df['Date'])  # Convert Date column to datetime
    pair_df.set_index('Date', inplace=True)
    pair_df['Ticker Pair'] = f'{ticker_1}-{ticker_2}'
    pair_df.columns = ['Spread', 'Ticker Pair']
    spread_list.append(pair_df[['Ticker Pair', 'Spread']])

spread_df = pd.concat(spread_list)

spread_df.to_csv("outputs/spreads.csv", index=True)

spread_df

,Ticker Pair,Spread
Date,,
2019-02-28,AMD-NVDA,19.700950
2019-03-31,AMD-NVDA,21.062952
2019-04-30,AMD-NVDA,23.137207
2019-05-31,AMD-NVDA,24.043767
2019-06-30,AMD-NVDA,26.288822
2019-07-31,AMD-NVDA,26.257243
2019-08-31,AMD-NVDA,27.283192
2019-09-30,AMD-NVDA,24.660011
2019-10-31,AMD-NVDA,28.929632
